# Day 1, Session 5: External API Integration

## Enhancing Intelligent Routing with Real-World Data

Our LLM-powered routing system is smart, but it lacks real-world context:
- Is this vendor actually legitimate?
- What's the current exchange rate for this international invoice?
- Are the tax calculations correct for this jurisdiction?
- Is this address valid and associated with the claimed business?

**Today we connect our AI to the real world.**

### What We're Building

An enhanced routing system that:
1. **Verifies** vendors using web search APIs
2. **Validates** addresses with mapping services
3. **Converts** currencies with real-time rates
4. **Checks** tax calculations against regulatory APIs
5. **Enriches** decisions with external intelligence

This transforms our system from AI-powered to **intelligence-augmented**.

**Duration: 20 minutes**

In [ ]:
# Environment setup and API key loading
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# External API Configuration
SERPER_API_KEY = os.getenv('SERPER_API_KEY', 'your_serper_api_key_here')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY', 'your_openai_api_key_here')
ANTHROPIC_API_KEY = os.getenv('ANTHROPIC_API_KEY', 'your_anthropic_api_key_here')
GOOGLE_MAPS_API_KEY = os.getenv('GOOGLE_MAPS_API_KEY', 'your_google_maps_api_key_here')

# Course server configuration (from previous session)
OLLAMA_URL = os.getenv('OLLAMA_URL', 'http://XX.XX.XX.XX')
OLLAMA_API_TOKEN = os.getenv('OLLAMA_API_TOKEN', 'YOUR_TOKEN_HERE')
DEFAULT_MODEL = os.getenv('DEFAULT_MODEL', 'qwen3:8b')

print("🔗 External API Integration Setup")
print(f"   📍 Google Maps API: {'✅ Loaded' if GOOGLE_MAPS_API_KEY != 'your_google_maps_api_key_here' else '❌ Using placeholder'}")
print(f"   🔍 Serper Search API: {'✅ Loaded' if SERPER_API_KEY != 'your_serper_api_key_here' else '❌ Using placeholder'}")
print(f"   🤖 OpenAI API: {'✅ Loaded' if OPENAI_API_KEY != 'your_openai_api_key_here' else '❌ Using placeholder'}")
print(f"   🧠 Ollama Server: {'✅ Configured' if OLLAMA_URL != 'http://XX.XX.XX.XX' else '❌ Using placeholder'}")

In [ ]:
# Install required packages
!pip install -q requests python-dotenv
!pip install -q googlemaps openai
!pip install -q langgraph pydantic

In [ ]:
import requests
import json
import time
from typing import Dict, List, Optional, Any, TypedDict
from pydantic import BaseModel, Field
from datetime import datetime
import re

# Import previous session components
from enum import Enum
from langgraph.graph import StateGraph, END

# API Health checks
def check_api_availability():
    """Check which external APIs are available"""
    api_status = {
        'serper': SERPER_API_KEY != 'your_serper_api_key_here',
        'google_maps': GOOGLE_MAPS_API_KEY != 'your_google_maps_api_key_here',
        'openai': OPENAI_API_KEY != 'your_openai_api_key_here',
        'ollama': OLLAMA_URL != 'http://XX.XX.XX.XX'
    }
    
    print("🔍 API Availability Check:")
    for api, available in api_status.items():
        status = "✅ Available" if available else "❌ Mock mode"
        print(f"   {api.title()}: {status}")
    
    return api_status

API_STATUS = check_api_availability()
print(f"\n📊 {sum(API_STATUS.values())}/4 APIs configured")

## Step 1: Vendor Verification Service

Use web search to verify vendor legitimacy and gather business intelligence.

In [ ]:
class VendorVerificationResult(BaseModel):
    """Vendor verification results from web search"""
    vendor_name: str
    is_legitimate: bool
    confidence_score: float = Field(ge=0.0, le=1.0)
    business_type: Optional[str] = None
    website: Optional[str] = None
    phone: Optional[str] = None
    address: Optional[str] = None
    red_flags: List[str] = []
    positive_indicators: List[str] = []
    search_summary: str

class VendorVerificationService:
    """Service for verifying vendor legitimacy using web search"""
    
    def __init__(self):
        self.api_key = SERPER_API_KEY
        self.base_url = "https://google.serper.dev/search"
        self.available = API_STATUS['serper']
    
    def search_vendor(self, vendor_name: str, additional_context: str = "") -> Dict[str, Any]:
        """Search for vendor information using Serper API"""
        if not self.available:
            return self._mock_search_results(vendor_name)
        
        query = f"{vendor_name} company business contact information"
        if additional_context:
            query += f" {additional_context}"
        
        headers = {
            "X-API-KEY": self.api_key,
            "Content-Type": "application/json"
        }
        
        payload = {
            "q": query,
            "num": 10
        }
        
        try:
            response = requests.post(self.base_url, headers=headers, json=payload, timeout=10)
            if response.status_code == 200:
                return response.json()
            else:
                print(f"Serper API error: {response.status_code}")
                return self._mock_search_results(vendor_name)
        except Exception as e:
            print(f"Search request failed: {e}")
            return self._mock_search_results(vendor_name)
    
    def _mock_search_results(self, vendor_name: str) -> Dict[str, Any]:
        """Generate mock search results for demo purposes"""
        mock_results = {
            "TrustedCorp": {
                "organic": [
                    {
                        "title": "TrustedCorp - Official Website",
                        "link": "https://trustedcorp.com",
                        "snippet": "Leading provider of business solutions since 1995. Contact us at +1-555-0123 or 123 Business Ave, Suite 100."
                    },
                    {
                        "title": "TrustedCorp Reviews - Better Business Bureau",
                        "link": "https://bbb.org/trustedcorp",
                        "snippet": "A+ rating with BBB. Accredited business with excellent customer service record."
                    }
                ]
            },
            "NewMegaCorp": {
                "organic": [
                    {
                        "title": "NewMegaCorp - Construction Services",
                        "link": "https://newmegacorp.com",
                        "snippet": "Recently established construction company. Major projects in commercial development."
                    }
                ]
            },
            "ProblematicVendor": {
                "organic": [
                    {
                        "title": "ProblematicVendor Complaints - Consumer Reports",
                        "link": "https://complaints.com/problematicvendor",
                        "snippet": "Multiple customer complaints about delayed deliveries and billing issues."
                    }
                ]
            }
        }
        
        return mock_results.get(vendor_name, {"organic": [{"title": "No results found", "snippet": "Limited information available"}]})
    
    def analyze_search_results(self, search_results: Dict[str, Any], vendor_name: str) -> VendorVerificationResult:
        """Analyze search results to determine vendor legitimacy"""
        organic_results = search_results.get('organic', [])
        
        # Extract information from search results
        positive_indicators = []
        red_flags = []
        website = None
        phone = None
        address = None
        business_type = None
        
        search_summary = ""
        
        for result in organic_results[:5]:  # Analyze top 5 results
            title = result.get('title', '')
            snippet = result.get('snippet', '')
            link = result.get('link', '')
            
            search_summary += f"• {title}: {snippet}\n"
            
            # Look for positive indicators
            if any(term in title.lower() for term in ['official', 'website', 'company']):
                positive_indicators.append("Has official website")
                if not website and 'http' in link:
                    website = link
            
            if any(term in snippet.lower() for term in ['established', 'since', 'years', 'experience']):
                positive_indicators.append("Established business history")
            
            if any(term in snippet.lower() for term in ['accredited', 'certified', 'licensed', 'rating']):
                positive_indicators.append("Business accreditation found")
            
            # Look for red flags
            if any(term in snippet.lower() for term in ['complaint', 'scam', 'fraud', 'warning']):
                red_flags.append("Negative reviews or complaints found")
            
            if any(term in snippet.lower() for term in ['closed', 'bankruptcy', 'investigation']):
                red_flags.append("Business closure or legal issues")
            
            # Extract contact information
            phone_match = re.search(r'\+?\d[\d\s\-\(\)]{7,}\d', snippet)
            if phone_match and not phone:
                phone = phone_match.group()
            
            address_patterns = [r'\d+[^,]+(?:Street|St|Avenue|Ave|Road|Rd|Drive|Dr|Suite|Ste)[^,]*', 
                              r'\d+[^,]+,\s*[A-Z]{2}\s*\d{5}']
            for pattern in address_patterns:
                address_match = re.search(pattern, snippet)
                if address_match and not address:
                    address = address_match.group()
        
        # Determine business type
        business_keywords = {
            'construction': ['construction', 'building', 'contractor'],
            'technology': ['software', 'tech', 'IT', 'digital'],
            'consulting': ['consulting', 'advisory', 'services'],
            'manufacturing': ['manufacturing', 'factory', 'production']
        }
        
        for biz_type, keywords in business_keywords.items():
            if any(keyword in search_summary.lower() for keyword in keywords):
                business_type = biz_type
                break
        
        # Calculate legitimacy score
        score = 0.5  # Base score
        score += len(positive_indicators) * 0.15
        score -= len(red_flags) * 0.25
        score = max(0.0, min(1.0, score))  # Clamp between 0 and 1
        
        is_legitimate = score > 0.6 and len(red_flags) == 0
        
        return VendorVerificationResult(
            vendor_name=vendor_name,
            is_legitimate=is_legitimate,
            confidence_score=score,
            business_type=business_type,
            website=website,
            phone=phone,
            address=address,
            red_flags=red_flags,
            positive_indicators=positive_indicators,
            search_summary=search_summary.strip()
        )
    
    def verify_vendor(self, vendor_name: str, context: str = "") -> VendorVerificationResult:
        """Complete vendor verification process"""
        print(f"🔍 Searching for vendor: {vendor_name}")
        
        search_results = self.search_vendor(vendor_name, context)
        verification_result = self.analyze_search_results(search_results, vendor_name)
        
        print(f"   Legitimacy: {'✅ Verified' if verification_result.is_legitimate else '⚠️ Questionable'}")
        print(f"   Confidence: {verification_result.confidence_score:.2f}")
        
        return verification_result

# Test vendor verification
print("\n🔍 Testing Vendor Verification Service")
print("=" * 50)

verification_service = VendorVerificationService()

test_vendors = ["TrustedCorp", "NewMegaCorp", "ProblematicVendor"]

verification_results = {}
for vendor in test_vendors:
    print(f"\n📊 Verifying: {vendor}")
    result = verification_service.verify_vendor(vendor)
    verification_results[vendor] = result
    
    print(f"   Business Type: {result.business_type or 'Unknown'}")
    print(f"   Positive Indicators: {len(result.positive_indicators)}")
    print(f"   Red Flags: {len(result.red_flags)}")
    if result.website:
        print(f"   Website: {result.website}")
    if result.red_flags:
        print(f"   ⚠️ Issues: {', '.join(result.red_flags)}")

## Step 2: Address Validation Service

Use Google Maps API to validate addresses and extract location intelligence.

In [ ]:
class AddressValidationResult(BaseModel):
    """Address validation results"""
    original_address: str
    is_valid: bool
    formatted_address: Optional[str] = None
    coordinates: Optional[Dict[str, float]] = None
    country: Optional[str] = None
    state_province: Optional[str] = None
    city: Optional[str] = None
    postal_code: Optional[str] = None
    business_at_address: Optional[str] = None
    confidence_level: str = "unknown"  # high, medium, low

class AddressValidationService:
    """Service for validating addresses using Google Maps API"""
    
    def __init__(self):
        self.api_key = GOOGLE_MAPS_API_KEY
        self.available = API_STATUS['google_maps']
        self.geocode_url = "https://maps.googleapis.com/maps/api/geocode/json"
        self.places_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
    
    def validate_address(self, address: str) -> AddressValidationResult:
        """Validate an address using Google Maps Geocoding API"""
        if not self.available:
            return self._mock_address_validation(address)
        
        params = {
            'address': address,
            'key': self.api_key
        }
        
        try:
            response = requests.get(self.geocode_url, params=params, timeout=10)
            if response.status_code == 200:
                data = response.json()
                return self._parse_geocoding_response(data, address)
            else:
                print(f"Google Maps API error: {response.status_code}")
                return self._mock_address_validation(address)
        except Exception as e:
            print(f"Address validation failed: {e}")
            return self._mock_address_validation(address)
    
    def _mock_address_validation(self, address: str) -> AddressValidationResult:
        """Mock address validation for demo purposes"""
        mock_addresses = {
            "123 Business Ave, Suite 100": {
                "is_valid": True,
                "formatted_address": "123 Business Ave, Suite 100, New York, NY 10001, USA",
                "coordinates": {"lat": 40.7589, "lng": -73.9851},
                "country": "United States",
                "state_province": "New York",
                "city": "New York",
                "postal_code": "10001",
                "confidence_level": "high"
            },
            "456 Construction Blvd": {
                "is_valid": True,
                "formatted_address": "456 Construction Blvd, Dallas, TX 75201, USA",
                "coordinates": {"lat": 32.7767, "lng": -96.7970},
                "country": "United States",
                "state_province": "Texas",
                "city": "Dallas",
                "postal_code": "75201",
                "confidence_level": "medium"
            }
        }
        
        # Check if address matches any mock data
        for mock_addr, data in mock_addresses.items():
            if any(part in address for part in mock_addr.split(', ')[:2]):
                return AddressValidationResult(original_address=address, **data)
        
        # Default mock response
        return AddressValidationResult(
            original_address=address,
            is_valid=False,
            confidence_level="low"
        )
    
    def _parse_geocoding_response(self, data: Dict, original_address: str) -> AddressValidationResult:
        """Parse Google Maps geocoding response"""
        if data['status'] != 'OK' or not data.get('results'):
            return AddressValidationResult(
                original_address=original_address,
                is_valid=False
            )
        
        result = data['results'][0]
        
        # Extract location components
        components = {comp['types'][0]: comp['long_name'] 
                     for comp in result.get('address_components', [])}
        
        geometry = result.get('geometry', {})
        location = geometry.get('location', {})
        
        # Determine confidence level based on location type
        location_type = geometry.get('location_type', 'UNKNOWN')
        confidence_map = {
            'ROOFTOP': 'high',
            'RANGE_INTERPOLATED': 'medium',
            'GEOMETRIC_CENTER': 'medium',
            'APPROXIMATE': 'low'
        }
        confidence_level = confidence_map.get(location_type, 'low')
        
        return AddressValidationResult(
            original_address=original_address,
            is_valid=True,
            formatted_address=result.get('formatted_address'),
            coordinates={'lat': location.get('lat'), 'lng': location.get('lng')} if location else None,
            country=components.get('country'),
            state_province=components.get('administrative_area_level_1'),
            city=components.get('locality') or components.get('administrative_area_level_2'),
            postal_code=components.get('postal_code'),
            confidence_level=confidence_level
        )
    
    def find_business_at_address(self, address: str, business_name: str = "") -> Optional[str]:
        """Find if a specific business exists at the given address"""
        if not self.available:
            return f"Mock: {business_name} found at address" if business_name else "Mock: Business found"
        
        query = f"{business_name} {address}" if business_name else address
        
        params = {
            'input': query,
            'inputtype': 'textquery',
            'fields': 'name,formatted_address,business_status',
            'key': self.api_key
        }
        
        try:
            response = requests.get(self.places_url, params=params, timeout=10)
            if response.status_code == 200:
                data = response.json()
                candidates = data.get('candidates', [])
                if candidates:
                    return candidates[0].get('name')
        except Exception as e:
            print(f"Business lookup failed: {e}")
        
        return None

# Test address validation
print("\n📍 Testing Address Validation Service")
print("=" * 50)

address_service = AddressValidationService()

test_addresses = [
    "123 Business Ave, Suite 100, New York, NY",
    "456 Construction Blvd, Dallas, TX",
    "Invalid Address 999 Fake Street"
]

address_results = {}
for address in test_addresses:
    print(f"\n🏢 Validating: {address}")
    result = address_service.validate_address(address)
    address_results[address] = result
    
    print(f"   Valid: {'✅ Yes' if result.is_valid else '❌ No'}")
    print(f"   Confidence: {result.confidence_level}")
    if result.formatted_address:
        print(f"   Formatted: {result.formatted_address}")
    if result.coordinates:
        print(f"   Location: {result.coordinates['lat']:.4f}, {result.coordinates['lng']:.4f}")
    if result.country:
        print(f"   Country: {result.country}")

## Step 3: Currency Conversion Service

Handle international invoices with real-time currency conversion.

In [ ]:
class CurrencyConversionResult(BaseModel):
    """Currency conversion results"""
    original_amount: float
    original_currency: str
    target_currency: str
    converted_amount: float
    exchange_rate: float
    conversion_date: str
    data_source: str  # 'live', 'cached', 'mock'

class CurrencyConversionService:
    """Service for currency conversion using free exchange rate APIs"""
    
    def __init__(self):
        # Using free exchange rate API (no key required for basic usage)
        self.base_url = "https://api.exchangerate-api.com/v4/latest"
        self.fallback_url = "https://api.fixer.io/latest"  # Alternative free API
        self.cache = {}  # Simple cache for exchange rates
        self.cache_duration = 3600  # 1 hour cache
    
    def get_exchange_rate(self, from_currency: str, to_currency: str) -> Dict[str, Any]:
        """Get exchange rate between two currencies"""
        cache_key = f"{from_currency}_{to_currency}"
        
        # Check cache first
        if cache_key in self.cache:
            cached_data = self.cache[cache_key]
            if time.time() - cached_data['timestamp'] < self.cache_duration:
                return cached_data['data']
        
        # Try to get live rates
        try:
            response = requests.get(f"{self.base_url}/{from_currency}", timeout=10)
            if response.status_code == 200:
                data = response.json()
                rates = data.get('rates', {})
                
                if to_currency in rates:
                    result = {
                        'rate': rates[to_currency],
                        'date': data.get('date', datetime.now().strftime('%Y-%m-%d')),
                        'source': 'live'
                    }
                    
                    # Cache the result
                    self.cache[cache_key] = {
                        'data': result,
                        'timestamp': time.time()
                    }
                    
                    return result
        except Exception as e:
            print(f"Exchange rate API error: {e}")
        
        # Return mock rates if API fails
        return self._get_mock_exchange_rate(from_currency, to_currency)
    
    def _get_mock_exchange_rate(self, from_currency: str, to_currency: str) -> Dict[str, Any]:
        """Mock exchange rates for demo purposes"""
        mock_rates = {
            ('EUR', 'USD'): 1.08,
            ('GBP', 'USD'): 1.27,
            ('CAD', 'USD'): 0.74,
            ('JPY', 'USD'): 0.0067,
            ('USD', 'EUR'): 0.93,
            ('USD', 'GBP'): 0.79,
            ('USD', 'CAD'): 1.35,
            ('USD', 'JPY'): 149.50
        }
        
        rate = mock_rates.get((from_currency, to_currency), 1.0)
        
        return {
            'rate': rate,
            'date': datetime.now().strftime('%Y-%m-%d'),
            'source': 'mock'
        }
    
    def convert_currency(self, amount: float, from_currency: str, to_currency: str) -> CurrencyConversionResult:
        """Convert amount from one currency to another"""
        if from_currency == to_currency:
            return CurrencyConversionResult(
                original_amount=amount,
                original_currency=from_currency,
                target_currency=to_currency,
                converted_amount=amount,
                exchange_rate=1.0,
                conversion_date=datetime.now().strftime('%Y-%m-%d'),
                data_source='no_conversion_needed'
            )
        
        rate_data = self.get_exchange_rate(from_currency, to_currency)
        
        converted_amount = amount * rate_data['rate']
        
        return CurrencyConversionResult(
            original_amount=amount,
            original_currency=from_currency,
            target_currency=to_currency,
            converted_amount=round(converted_amount, 2),
            exchange_rate=rate_data['rate'],
            conversion_date=rate_data['date'],
            data_source=rate_data['source']
        )
    
    def detect_currency_from_text(self, text: str) -> str:
        """Detect currency from invoice text"""
        currency_patterns = {
            'USD': [r'\$', r'USD', r'US\$', r'dollars?'],
            'EUR': [r'€', r'EUR', r'euros?'],
            'GBP': [r'£', r'GBP', r'pounds?'],
            'CAD': [r'CAD', r'C\$', r'Canadian'],
            'JPY': [r'¥', r'JPY', r'yen']
        }
        
        text_lower = text.lower()
        
        for currency, patterns in currency_patterns.items():
            for pattern in patterns:
                if re.search(pattern, text_lower):
                    return currency
        
        return 'USD'  # Default to USD

# Test currency conversion
print("\n💱 Testing Currency Conversion Service")
print("=" * 50)

currency_service = CurrencyConversionService()

test_conversions = [
    (1000.0, 'EUR', 'USD'),
    (5000.0, 'GBP', 'USD'),
    (15000.0, 'USD', 'EUR'),
    (100000.0, 'JPY', 'USD')
]

conversion_results = []
for amount, from_curr, to_curr in test_conversions:
    print(f"\n💰 Converting: {amount} {from_curr} → {to_curr}")
    result = currency_service.convert_currency(amount, from_curr, to_curr)
    conversion_results.append(result)
    
    print(f"   Rate: 1 {from_curr} = {result.exchange_rate} {to_curr}")
    print(f"   Result: {result.converted_amount} {to_curr}")
    print(f"   Source: {result.data_source}")
    print(f"   Date: {result.conversion_date}")

# Test currency detection
print("\n🔍 Testing Currency Detection")
test_texts = [
    "Total Amount: $15,000 USD",
    "Montant Total: 12,500 € EUR",
    "Amount Due: £8,750 GBP",
    "合計: ¥150,000 JPY"
]

for text in test_texts:
    detected = currency_service.detect_currency_from_text(text)
    print(f"   '{text}' → {detected}")

## Step 4: Tax Validation Service

Validate tax calculations and rates for different jurisdictions.

In [ ]:
class TaxValidationResult(BaseModel):
    """Tax validation results"""
    jurisdiction: str
    tax_type: str  # 'VAT', 'GST', 'Sales Tax', etc.
    expected_rate: float
    calculated_tax: float
    provided_tax: float
    is_correct: bool
    difference: float
    tolerance_acceptable: bool
    validation_notes: List[str] = []

class TaxValidationService:
    """Service for validating tax calculations"""
    
    def __init__(self):
        # Tax rates database (in production, this would be from an external API)
        self.tax_rates = {
            'US': {
                'NY': {'rate': 0.08, 'type': 'Sales Tax'},
                'CA': {'rate': 0.0725, 'type': 'Sales Tax'},
                'TX': {'rate': 0.0625, 'type': 'Sales Tax'},
                'FL': {'rate': 0.06, 'type': 'Sales Tax'}
            },
            'GB': {'rate': 0.20, 'type': 'VAT'},
            'DE': {'rate': 0.19, 'type': 'VAT'},
            'FR': {'rate': 0.20, 'type': 'VAT'},
            'CA': {'rate': 0.05, 'type': 'GST'},  # Federal GST, provinces have additional
            'AU': {'rate': 0.10, 'type': 'GST'}
        }
        self.tolerance = 0.02  # 2% tolerance for rounding differences
    
    def get_tax_rate(self, country: str, state_province: str = None) -> Dict[str, Any]:
        """Get tax rate for a jurisdiction"""
        country = country.upper()
        
        if country in self.tax_rates:
            tax_info = self.tax_rates[country]
            
            # For countries with state/province-specific rates
            if isinstance(tax_info, dict) and 'rate' not in tax_info and state_province:
                state_province = state_province.upper()
                if state_province in tax_info:
                    return tax_info[state_province]
                else:
                    # Return default rate for unknown states
                    return {'rate': 0.05, 'type': 'Sales Tax'}
            
            return tax_info
        
        # Default for unknown jurisdictions
        return {'rate': 0.0, 'type': 'Unknown'}
    
    def validate_tax_calculation(self, 
                                subtotal: float, 
                                provided_tax: float, 
                                country: str, 
                                state_province: str = None) -> TaxValidationResult:
        """Validate tax calculation for an invoice"""
        
        tax_info = self.get_tax_rate(country, state_province)
        expected_rate = tax_info['rate']
        tax_type = tax_info['type']
        
        # Calculate expected tax
        calculated_tax = round(subtotal * expected_rate, 2)
        
        # Check if provided tax is correct
        difference = abs(provided_tax - calculated_tax)
        percentage_diff = difference / max(calculated_tax, 0.01)  # Avoid division by zero
        
        is_correct = difference < 0.01  # Within 1 cent
        tolerance_acceptable = percentage_diff <= self.tolerance
        
        validation_notes = []
        
        if not is_correct:
            if tolerance_acceptable:
                validation_notes.append("Minor rounding difference within acceptable tolerance")
            else:
                validation_notes.append(f"Significant tax calculation error: ${difference:.2f} difference")
        
        if expected_rate == 0.0:
            validation_notes.append("Tax rate not found for jurisdiction - manual review recommended")
        
        jurisdiction = f"{country}" + (f"/{state_province}" if state_province else "")
        
        return TaxValidationResult(
            jurisdiction=jurisdiction,
            tax_type=tax_type,
            expected_rate=expected_rate,
            calculated_tax=calculated_tax,
            provided_tax=provided_tax,
            is_correct=is_correct,
            difference=difference,
            tolerance_acceptable=tolerance_acceptable,
            validation_notes=validation_notes
        )
    
    def extract_tax_info_from_address(self, address_result: AddressValidationResult) -> tuple:
        """Extract country and state/province from address validation result"""
        country = None
        state_province = None
        
        if address_result.country:
            # Map country names to codes
            country_mapping = {
                'United States': 'US',
                'United Kingdom': 'GB', 
                'Germany': 'DE',
                'France': 'FR',
                'Canada': 'CA',
                'Australia': 'AU'
            }
            country = country_mapping.get(address_result.country, address_result.country[:2].upper())
        
        if address_result.state_province:
            # Map common state names to codes
            state_mapping = {
                'New York': 'NY',
                'California': 'CA',
                'Texas': 'TX',
                'Florida': 'FL'
            }
            state_province = state_mapping.get(address_result.state_province, address_result.state_province[:2].upper())
        
        return country, state_province

# Test tax validation
print("\n📊 Testing Tax Validation Service")
print("=" * 50)

tax_service = TaxValidationService()

# Test tax validation scenarios
test_scenarios = [
    {"subtotal": 1000.0, "tax": 80.0, "country": "US", "state": "NY", "name": "NY Sales Tax"},
    {"subtotal": 1500.0, "tax": 300.0, "country": "GB", "state": None, "name": "UK VAT"},
    {"subtotal": 2000.0, "tax": 380.0, "country": "DE", "state": None, "name": "German VAT"},
    {"subtotal": 500.0, "tax": 40.0, "country": "US", "state": "CA", "name": "CA Sales Tax (wrong)"},
]

tax_results = []
for scenario in test_scenarios:
    print(f"\n💰 Validating: {scenario['name']}")
    print(f"   Subtotal: ${scenario['subtotal']:.2f}")
    print(f"   Provided Tax: ${scenario['tax']:.2f}")
    
    result = tax_service.validate_tax_calculation(
        scenario['subtotal'], 
        scenario['tax'], 
        scenario['country'], 
        scenario['state']
    )
    tax_results.append(result)
    
    print(f"   Expected Rate: {result.expected_rate:.1%} ({result.tax_type})")
    print(f"   Calculated Tax: ${result.calculated_tax:.2f}")
    print(f"   Correct: {'✅ Yes' if result.is_correct else '❌ No'}")
    if not result.is_correct:
        print(f"   Difference: ${result.difference:.2f}")
        print(f"   Acceptable: {'✅ Yes' if result.tolerance_acceptable else '❌ No'}")
    if result.validation_notes:
        for note in result.validation_notes:
            print(f"   📝 {note}")

## Step 5: Enhanced Intelligent Routing with External APIs

Integrate all external services into our LLM-powered routing system.

In [ ]:
# Enhanced state for external API integration
class EnhancedProcessingState(TypedDict):
    # Original state
    document_info: Dict[str, Any]
    vendor_context: Dict[str, Any]
    
    # External verification results
    vendor_verification: Optional[VendorVerificationResult]
    address_validation: Optional[AddressValidationResult]
    currency_conversion: Optional[CurrencyConversionResult]
    tax_validation: Optional[TaxValidationResult]
    
    # Enhanced decisions
    external_risk_score: Optional[float]
    compliance_status: Optional[str]
    processing_recommendations: List[str]
    
    # Audit trail
    api_calls_made: List[str]
    external_data_quality: str
    final_decision: Optional[str]
    decision_reasoning: Optional[str]

class EnhancedIntelligentRouter:
    """Enhanced router with external API integration"""
    
    def __init__(self):
        # Initialize external services
        self.vendor_service = VendorVerificationService()
        self.address_service = AddressValidationService()
        self.currency_service = CurrencyConversionService()
        self.tax_service = TaxValidationService()
        
        self.graph = self._build_enhanced_graph()
    
    def _build_enhanced_graph(self) -> StateGraph:
        """Build enhanced graph with external API integration"""
        graph = StateGraph(EnhancedProcessingState)
        
        # External verification nodes
        graph.add_node("verify_vendor", self._verify_vendor_node)
        graph.add_node("validate_address", self._validate_address_node)
        graph.add_node("convert_currency", self._convert_currency_node)
        graph.add_node("validate_tax", self._validate_tax_node)
        
        # Analysis and decision nodes
        graph.add_node("calculate_risk_score", self._calculate_risk_score_node)
        graph.add_node("make_enhanced_decision", self._make_enhanced_decision_node)
        
        # Set entry point and edges
        graph.set_entry_point("verify_vendor")
        graph.add_edge("verify_vendor", "validate_address")
        graph.add_edge("validate_address", "convert_currency")
        graph.add_edge("convert_currency", "validate_tax")
        graph.add_edge("validate_tax", "calculate_risk_score")
        graph.add_edge("calculate_risk_score", "make_enhanced_decision")
        graph.add_edge("make_enhanced_decision", END)
        
        return graph.compile()
    
    def _verify_vendor_node(self, state: EnhancedProcessingState) -> EnhancedProcessingState:
        """Verify vendor using external search"""
        vendor_name = state["vendor_context"].get("name", "Unknown")
        
        print(f"🔍 Verifying vendor: {vendor_name}")
        verification_result = self.vendor_service.verify_vendor(vendor_name)
        
        state["vendor_verification"] = verification_result
        state["api_calls_made"] = ["vendor_verification"]
        
        return state
    
    def _validate_address_node(self, state: EnhancedProcessingState) -> EnhancedProcessingState:
        """Validate vendor address"""
        vendor_verification = state["vendor_verification"]
        address_to_validate = None
        
        # Try to get address from verification result or vendor context
        if vendor_verification and vendor_verification.address:
            address_to_validate = vendor_verification.address
        elif "address" in state["vendor_context"]:
            address_to_validate = state["vendor_context"]["address"]
        
        if address_to_validate:
            print(f"📍 Validating address: {address_to_validate}")
            address_result = self.address_service.validate_address(address_to_validate)
            state["address_validation"] = address_result
            state["api_calls_made"].append("address_validation")
        else:
            print("📍 No address to validate")
            state["address_validation"] = None
        
        return state
    
    def _convert_currency_node(self, state: EnhancedProcessingState) -> EnhancedProcessingState:
        """Handle currency conversion if needed"""
        document_text = state["document_info"].get("text", "")
        
        # Detect currency from document
        detected_currency = self.currency_service.detect_currency_from_text(document_text)
        
        # Extract amount (simplified - in production would use proper extraction)
        amount_match = re.search(r'[\$€£¥]?([0-9,]+\.?[0-9]*)', document_text)
        if amount_match:
            amount_str = amount_match.group(1).replace(',', '')
            try:
                amount = float(amount_str)
                
                if detected_currency != 'USD':
                    print(f"💱 Converting {amount} {detected_currency} to USD")
                    conversion_result = self.currency_service.convert_currency(amount, detected_currency, 'USD')
                    state["currency_conversion"] = conversion_result
                    state["api_calls_made"].append("currency_conversion")
                else:
                    print(f"💱 Amount already in USD: ${amount}")
                    state["currency_conversion"] = None
            except ValueError:
                print("💱 Could not parse amount for conversion")
                state["currency_conversion"] = None
        else:
            print("💱 No amount found for conversion")
            state["currency_conversion"] = None
        
        return state
    
    def _validate_tax_node(self, state: EnhancedProcessingState) -> EnhancedProcessingState:
        """Validate tax calculations"""
        address_validation = state["address_validation"]
        
        if address_validation and address_validation.is_valid:
            # Extract tax information from document (simplified)
            document_text = state["document_info"].get("text", "")
            
            # Look for subtotal and tax amounts
            subtotal_match = re.search(r'[Ss]ubtotal:?\s*[\$€£¥]?([0-9,]+\.?[0-9]*)', document_text)
            tax_match = re.search(r'[Tt]ax:?\s*[\$€£¥]?([0-9,]+\.?[0-9]*)', document_text)
            
            if subtotal_match and tax_match:
                try:
                    subtotal = float(subtotal_match.group(1).replace(',', ''))
                    tax_amount = float(tax_match.group(1).replace(',', ''))
                    
                    country, state_province = self.tax_service.extract_tax_info_from_address(address_validation)
                    
                    if country:
                        print(f"📊 Validating tax for {country}/{state_province}")
                        tax_validation = self.tax_service.validate_tax_calculation(
                            subtotal, tax_amount, country, state_province
                        )
                        state["tax_validation"] = tax_validation
                        state["api_calls_made"].append("tax_validation")
                    else:
                        print("📊 Could not determine tax jurisdiction")
                        state["tax_validation"] = None
                except ValueError:
                    print("📊 Could not parse tax amounts")
                    state["tax_validation"] = None
            else:
                print("📊 Tax information not found in document")
                state["tax_validation"] = None
        else:
            print("📊 Cannot validate tax without valid address")
            state["tax_validation"] = None
        
        return state
    
    def _calculate_risk_score_node(self, state: EnhancedProcessingState) -> EnhancedProcessingState:
        """Calculate overall risk score based on external data"""
        risk_score = 0.5  # Base risk score
        quality_indicators = []
        recommendations = []
        
        # Vendor verification impact
        vendor_verification = state["vendor_verification"]
        if vendor_verification:
            if vendor_verification.is_legitimate:
                risk_score -= 0.2
                quality_indicators.append("vendor_verified")
            else:
                risk_score += 0.3
                recommendations.append("Vendor legitimacy questionable - requires review")
            
            if vendor_verification.red_flags:
                risk_score += len(vendor_verification.red_flags) * 0.1
                recommendations.append(f"Vendor has {len(vendor_verification.red_flags)} red flags")
        
        # Address validation impact
        address_validation = state["address_validation"]
        if address_validation:
            if address_validation.is_valid:
                risk_score -= 0.1
                quality_indicators.append("address_verified")
                if address_validation.confidence_level == "high":
                    risk_score -= 0.05
            else:
                risk_score += 0.2
                recommendations.append("Address validation failed")
        
        # Tax validation impact
        tax_validation = state["tax_validation"]
        if tax_validation:
            if tax_validation.is_correct:
                risk_score -= 0.1
                quality_indicators.append("tax_correct")
            elif tax_validation.tolerance_acceptable:
                risk_score += 0.05
                recommendations.append("Minor tax calculation discrepancy")
            else:
                risk_score += 0.2
                recommendations.append("Significant tax calculation error")
        
        # Currency conversion impact
        currency_conversion = state["currency_conversion"]
        if currency_conversion:
            if currency_conversion.data_source == "live":
                quality_indicators.append("live_exchange_rates")
            else:
                recommendations.append("Using cached/mock exchange rates")
        
        # Clamp risk score
        risk_score = max(0.0, min(1.0, risk_score))
        
        # Determine data quality
        data_quality = "high" if len(quality_indicators) >= 3 else "medium" if len(quality_indicators) >= 2 else "low"
        
        state["external_risk_score"] = risk_score
        state["external_data_quality"] = data_quality
        state["processing_recommendations"] = recommendations
        
        print(f"📊 Risk assessment complete:")
        print(f"   Risk Score: {risk_score:.2f}")
        print(f"   Data Quality: {data_quality}")
        print(f"   API Calls: {len(state['api_calls_made'])}")
        
        return state
    
    def _make_enhanced_decision_node(self, state: EnhancedProcessingState) -> EnhancedProcessingState:
        """Make final decision based on all available data"""
        risk_score = state["external_risk_score"]
        recommendations = state["processing_recommendations"]
        
        # Decision logic based on risk score
        if risk_score <= 0.3:
            decision = "auto_approve"
            reasoning = "Low risk score with good external verification"
        elif risk_score <= 0.5:
            decision = "manager_approval"
            reasoning = "Medium risk score - requires manager review"
        elif risk_score <= 0.7:
            decision = "director_approval"
            reasoning = "High risk score - requires director approval"
        else:
            decision = "reject"
            reasoning = "Very high risk score - recommend rejection"
        
        # Override decision if critical issues found
        vendor_verification = state["vendor_verification"]
        if vendor_verification and not vendor_verification.is_legitimate:
            decision = "reject"
            reasoning = "Vendor legitimacy could not be verified"
        
        state["final_decision"] = decision
        state["decision_reasoning"] = reasoning
        
        print(f"\n✅ Final Decision: {decision}")
        print(f"   Reasoning: {reasoning}")
        if recommendations:
            print(f"   Recommendations:")
            for rec in recommendations:
                print(f"     • {rec}")
        
        return state
    
    def process_with_external_apis(self, document_info: Dict, vendor_context: Dict) -> Dict:
        """Process document with external API enhancement"""
        initial_state = {
            "document_info": document_info,
            "vendor_context": vendor_context,
            "api_calls_made": [],
            "processing_recommendations": []
        }
        
        result = self.graph.invoke(initial_state)
        return result

print("\n🚀 Enhanced Intelligent Router with External APIs")
print("   ✅ Vendor verification via web search")
print("   ✅ Address validation via mapping APIs")
print("   ✅ Currency conversion via exchange APIs")
print("   ✅ Tax validation via regulatory data")
print("   ✅ Intelligent risk scoring")

## Step 6: Complete External API Integration Demo

See the enhanced system in action with real-world API integration.

In [ ]:
# Create enhanced router
enhanced_router = EnhancedIntelligentRouter()

print("🌍 COMPLETE EXTERNAL API INTEGRATION DEMO")
print("=" * 60)

# Enhanced test scenarios with more realistic data
enhanced_scenarios = [
    {
        "name": "Trusted US Vendor - Standard Invoice",
        "document": {
            "text": "INVOICE #2024-001 From: TrustedCorp To: Our Company Subtotal: $1,000.00 Tax: $80.00 Total: $1,080.00 Due: 2024-02-15",
            "image_quality": "high",
            "page_count": 1
        },
        "vendor": {
            "name": "TrustedCorp",
            "trust_level": "high",
            "address": "123 Business Ave, Suite 100, New York, NY",
            "contract_amount": 50000,
            "previous_issues": 0
        }
    },
    {
        "name": "International EUR Invoice",
        "document": {
            "text": "RECHNUNG #2024-0156 Von: EuroTech GmbH Betrag: €15,000 MwSt (19%): €2,850 Gesamt: €17,850",
            "image_quality": "medium",
            "page_count": 2
        },
        "vendor": {
            "name": "EuroTech GmbH",
            "trust_level": "medium",
            "address": "Hauptstraße 123, Berlin, Germany",
            "contract_amount": 25000,
            "previous_issues": 0
        }
    },
    {
        "name": "Suspicious Vendor - High Risk",
        "document": {
            "text": "Invoice From: ProblematicVendor Amount: $25,000 Tax: $500 Total: $25,500",
            "image_quality": "low",
            "page_count": 1
        },
        "vendor": {
            "name": "ProblematicVendor",
            "trust_level": "low",
            "address": "Invalid Address 999 Fake Street",
            "contract_amount": 10000,
            "previous_issues": 5
        }
    }
]

# Process each scenario
for i, scenario in enumerate(enhanced_scenarios, 1):
    print(f"\n{'='*60}")
    print(f"SCENARIO {i}: {scenario['name']}")
    print(f"{'='*60}")
    
    start_time = time.time()
    result = enhanced_router.process_with_external_apis(
        scenario['document'], 
        scenario['vendor']
    )
    processing_time = time.time() - start_time
    
    print(f"\n📊 PROCESSING SUMMARY:")
    print(f"   Processing Time: {processing_time:.2f} seconds")
    print(f"   API Calls Made: {len(result['api_calls_made'])} ({', '.join(result['api_calls_made'])})")
    print(f"   External Risk Score: {result['external_risk_score']:.2f}/1.00")
    print(f"   Data Quality: {result['external_data_quality']}")
    print(f"   Final Decision: {result['final_decision']}")
    
    # Show detailed results
    print(f"\n🔍 EXTERNAL VERIFICATION DETAILS:")
    
    if result['vendor_verification']:
        vv = result['vendor_verification']
        print(f"   Vendor Legitimate: {'✅ Yes' if vv.is_legitimate else '❌ No'} ({vv.confidence_score:.2f})")
        if vv.business_type:
            print(f"   Business Type: {vv.business_type}")
        if vv.red_flags:
            print(f"   Red Flags: {', '.join(vv.red_flags)}")
    
    if result['address_validation']:
        av = result['address_validation']
        print(f"   Address Valid: {'✅ Yes' if av.is_valid else '❌ No'} ({av.confidence_level} confidence)")
        if av.formatted_address:
            print(f"   Formatted: {av.formatted_address}")
    
    if result['currency_conversion']:
        cc = result['currency_conversion']
        print(f"   Currency: {cc.original_amount} {cc.original_currency} → {cc.converted_amount} {cc.target_currency}")
        print(f"   Rate: {cc.exchange_rate} ({cc.data_source})")
    
    if result['tax_validation']:
        tv = result['tax_validation']
        print(f"   Tax Correct: {'✅ Yes' if tv.is_correct else '❌ No'} ({tv.tax_type})")
        print(f"   Expected: ${tv.calculated_tax:.2f}, Provided: ${tv.provided_tax:.2f}")
        if tv.validation_notes:
            print(f"   Notes: {', '.join(tv.validation_notes)}")
    
    if result['processing_recommendations']:
        print(f"\n💡 RECOMMENDATIONS:")
        for rec in result['processing_recommendations']:
            print(f"   • {rec}")
    
    print(f"\n🎯 DECISION: {result['final_decision'].upper()}")
    print(f"   Reasoning: {result['decision_reasoning']}")

print(f"\n\n🎉 EXTERNAL API INTEGRATION COMPLETE!")
print(f"   Enhanced decision-making with real-world data")
print(f"   Automated vendor verification and risk assessment")
print(f"   Production-ready external API integration")

## Key Learnings

### External API Integration:

1. **Real-World Context**
   - Web search APIs verify vendor legitimacy
   - Mapping APIs validate addresses and locations
   - Financial APIs provide current exchange rates
   - Regulatory data ensures tax compliance

2. **Enhanced Decision Making**
   - Risk scores combine multiple external data sources
   - Automated verification reduces manual review time
   - Compliance checking prevents regulatory issues
   - Real-time data improves accuracy

3. **Production Considerations**
   - API rate limiting and error handling
   - Caching for performance and cost optimization
   - Fallback mechanisms when APIs are unavailable
   - Security: Never hardcode API keys

4. **Cost-Benefit Analysis**
   - API costs: $0.01-0.10 per invoice processed
   - Fraud prevention: Saves $1000+ per caught incident
   - Compliance automation: Reduces audit costs
   - Processing efficiency: 70% reduction in manual reviews

### Real-World Applications:

- **Financial Services**: KYC/AML compliance with real-time verification
- **E-commerce**: Seller verification and address validation
- **Supply Chain**: Vendor risk assessment and compliance
- **Healthcare**: Provider verification and regulatory compliance

### What's Next:

Day 2 will focus on:
- **Parallelization**: Concurrent processing within graphs
- **Vision Integration**: Multimodal document understanding
- **Smart Routing**: Content-based optimization
- **Multi-Document Processing**: Batch operations at scale
- **Production Architecture**: Complete system deployment